In [ ]:
from qcrypto.agents import Agent
from qcrypto.qstates import QstateEnt, QstateUnEnt
from qcrypto.gates import H_gate
import numpy as np



def Agent_profile():
    Alice = Agent()
    Alice.set_qstate(qstate=public_qstate,qstate_type='public')
    Alice_base_choice = np.random.choice([True, False], size=(numqubits))
    Alice_to_gate = np.nonzero(Alice_base_choice)[0]
    Alice.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Alice_to_gate)
    Alice.get_key(qstate_type="public")
    Alice.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Alice_to_gate)

In [ ]:
def BB84(numqubits: int, numcheckbits: int, eve: bool = False):
    """
    Implements the BB84 protocol using qcrypto.

    Args:
        - numqubits (int): Number of qubits Alice will send to Bob
        - numcheckbits (int): Number of bits Alice and Bob will compare through the classical channel
        - eve (bool): Whether or not Eve will intrude in Alice and Bob's key exchange

    Returns:
        Whether or not Eve was discovered, as well as Alice's and Bob's keys at the end of the protocol
    """

    # Initializations

    public_qstate = QstateUnEnt(init_method="random", num_qubits=numqubits)

    Alice = Agent()
    Alice.set_qstate(qstate=public_qstate, qstate_type="public")
    Bob = Agent()
    Bob.set_qstate(qstate=public_qstate, qstate_type="public")
    if eve:
        Eve = Agent()
        Eve.set_qstate(qstate=public_qstate, qstate_type="public")

    # Protocol
    
    Alice_base_choice = np.random.choice([True, False], size=(numqubits))
    Alice_to_gate = np.nonzero(Alice_base_choice)[0]
    Alice.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Alice_to_gate)
    Alice.get_key(qstate_type="public")
    Alice.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Alice_to_gate)

    if eve:
        Eve_base_choice = np.random.choice([True, False], size=(numqubits))
        Eve_to_gate = np.nonzero(Eve_base_choice)[0]
        Eve.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Eve_to_gate)
        Eve.get_key(qstate_type="public")
        Eve.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Eve_to_gate)

    Bob_base_choice = np.random.choice([True, False], size=(numqubits))
    Bob_to_gate = np.nonzero(Bob_base_choice)[0]
    Bob.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Bob_to_gate)
    Bob.get_key(qstate_type="public")
    Bob.apply_gate(qstate_type="public", gate=H_gate, qubit_idx=Bob_to_gate)

    # Throwing away bits with diff bases
    diff_base_mask = Alice_base_choice == Bob_base_choice

    Alice.keys["public"] = Alice.keys["public"][diff_base_mask]
    Bob.keys["public"] = Bob.keys["public"][diff_base_mask]

    # Alice and Bob communicate check bits 
    Alice_checkbits = Alice.keys["public"][:numcheckbits]
    Bob_checkbits = Bob.keys["public"][:numcheckbits]
    
    Alice.keys["public"] = Alice.keys["public"][numcheckbits:]
    Bob.keys["public"] = Bob.keys["public"][numcheckbits:]

    discovered = ~np.all(Alice_checkbits == Bob_checkbits)

    return discovered, (Alice.keys["public"], Bob.keys["public"])



In [ ]:
num_qubits = 30
num_checks = numqubits//2

public_qstate = QstateUnEnt(init_method="random", num_qubits=numqubits)


In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -u 0.000_001 -T data/BB84_profile.txt -f BB84 BB84(num_qubits, num_checks, eve=True)

In [ ]:
# Running in units of µs
%lprun -u 0.000_001 -T data/Agent_profile.txt -f Agent_profile Agent_profile()

In [ ]:
%lprun -u 0.000_001 -T data/qstate_measure_all_profile.txt -f public_qstate.measure_all public_qstate.measure_all() 

In [ ]:
%lprun -u 0.000_001 -T data/qstate_measure_profile.txt -f public_qstate.measure [public_qstate.measure(i) for i in range(numqubits) ]

The issue seems to be Numpy's `rng.choice` method

In [ ]:
rng = np.random.default_rng()

In [ ]:
%%timeit
rng.choice([0,1],p = [.02,.98])

There's not much more to improve here now. We would need to either make a faster implementation of NumPy's `rng.choice` method or try Numba.
However Numba does not support type annotations yet so will leave it as is.